In [53]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep, strftime
from random import randint
import pandas as pd

chromedriver_path = 'C:/Users/fitzg/Downloads/chromedriver_win32/chromedriver.exe' # Change this to your own chromedriver path!
webdriver = webdriver.Chrome(executable_path=chromedriver_path)
sleep(2)
webdriver.get('https://www.instagram.com/accounts/login/?source=auth_switcher')
sleep(3)

username = webdriver.find_element_by_name('username')
username.send_keys('')
password = webdriver.find_element_by_name('password')
password.send_keys('')

button_login = webdriver.find_element_by_css_selector('#react-root > section > main > div > article > div > div:nth-child(1) > div > form > div:nth-child(4) > button > div')
button_login.click()
sleep(3)

notnow = webdriver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm')
notnow.click() #comment these last 2 lines out, if you don't get a pop up asking about notifications

In [58]:
hashtag_list = ['illustration']
location_list = []

comment_list = pd.read_csv('comments_list.csv', delimiter=',').iloc[:,1:2]
comment_list = list(comment_list['0'].fillna('a'))
comment_list=["\n".join(i.splitlines()) for i in comment_list]
prev_user_list = pd.read_csv('users_followed_list.csv', delimiter=',').iloc[:,1:2]
prev_user_list = list(prev_user_list['0'])
follower_list = pd.read_csv('follower_list.csv', delimiter=',').iloc[:,1:2]
follower_list = list(follower_list['0'])

new_followed = []
new_comment = []
tag = -1
followed = 0
likes = 0
comments = 0


In [34]:
# for location in location_list:
#     location += 1
#     webdriver.get('https://www.instagram.com/explore/locations/'+ location_list[location] + '/')
#     sleep(5)
#     first_thumbnail = webdriver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div')
    
#     first_thumbnail.click()
#     sleep(randint(1,2)) 

In [59]:
#search by hashtag
for hashtag in hashtag_list:
    tag += 1
    webdriver.get('https://www.instagram.com/explore/tags/'+ hashtag_list[tag] + '/')
    sleep(5)
    first_thumbnail = webdriver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div')
    
    first_thumbnail.click()
    sleep(randint(1,2))    
    try:        
        for x in range(1,300):
            try:
                webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/header/div[2]/div[1]/div[1]/h2')
                
                username = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/header/div[2]/div[1]/div[1]/h2').text
                #if 10 seconds pass and cant find username then skip
                comment = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span').text

                #comments for users already posted to
                if username in prev_user_list:

                    #makes sure not commenting on same post
                    if comment not in comment_list:
                        new_comment.append(comment)

                        button_like = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[1]/span[1]/button/span')       
                        button_like.click()
                        sleep(randint(5,10))
                        try:
                            webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
                        except:
                            pass
                        likes += 1
                        sleep(randint(25,30))
                        
                        likecount=webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[2]/div/div/button').text[:-5]
                        likecount1=int(float(likecount.replace(',','')))
                        if likecount1 > 100:
                            comm_prob = randint(1,8)
                            if comm_prob < 4:
                                comments += 1
                                webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[1]/span[2]/button/span').click()
                                comment_box = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[3]/div/form/textarea')

                                if comm_prob ==1:
                                    comment_box.send_keys('Really cool @'+username+' !')
                                elif comm_prob ==2:
                                    comment_box.send_keys('Great work @'+username+' !!')
                                elif comm_prob == 3:
                                    comment_box.send_keys('Amazing @'+username+' !!!')

                                sleep(1)
                                post_comment = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[3]/div/form/button')
                                post_comment.click()
                                try:
                                    webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
                                except:
                                    pass
                                sleep(randint(25,30))

                            webdriver.find_element_by_link_text('Next').click()
                            sleep(randint(25,30)) 
                        else:
                            webdriver.find_element_by_link_text('Next').click()
                            sleep(randint(25,30))
                    else:
                        webdriver.find_element_by_link_text('Next').click()
                        sleep(randint(25,30))

                elif username not in prev_user_list:
                    
                    new_followed.append(username)
                    new_comment.append(comment)
                    likecount=webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[2]/div/div/button').text[:-6]
                    likecount1=int(float(likecount.replace(',','')))
                    
                    if (likecount1 >= 50) & (likecount1 < 1000):
                        if webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/header/div[2]/div[1]/div[2]/button').text == 'Follow':
                            webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/header/div[2]/div[1]/div[2]/button').click()
                            followed += 1
                            try:
                                webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
                            except:
                                pass
                        button_like = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[1]/span[1]/button/span')       
                        button_like.click()
                        sleep(randint(5,10))
                        try:
                            webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
                        except:
                            pass
                        likes += 1
                        sleep(randint(25,30))

                        comm_prob = randint(1,12)
                        if comm_prob < 4:
                            comments += 1
                            webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[1]/span[2]/button/span').click()
                            comment_box = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[3]/div/form/textarea')

                            if (comm_prob ==5):
                                if randint(1,2)==2:
                                    comment_box.send_keys('Really cool @'+username+' ! Check out my art account, new to IG! Much appreciated!')
                                else:
                                    comment_box.send_keys('Great work @'+username+' ! Just created my art profile, please check it out! Thank you!')
                            if (comm_prob ==4):
                                if randint(1,2)==2:
                                    comment_box.send_keys('Great work @'+username+' ! Just created my art profile, please check it out, trying to get it off the ground! Thank you!')
                                else:
                                    comment_box.send_keys('I like your work @'+username+' ! Check out my art account, I just started it!')
                            if (comm_prob ==3):
                                if randint(1,2)==2:
                                    comment_box.send_keys('Awesome work @'+username+' ! Please check my art account, trying to get my page off the ground!!')
                                else:
                                    comment_box.send_keys('I like your art @'+username+' ! Check out my art profile, new to IG! Much appreciated!')
                            if (comm_prob ==2):
                                if randint(1,2)==2:
                                    comment_box.send_keys('This is great @'+username+' keep up the good work! Please check out my art account, new to IG trying to get my page up off the ground!!!')
                                else:
                                    comment_box.send_keys('Really nice work @'+username+' ! Just created my art profile! Thank you!')
                            if (comm_prob ==1):
                                if randint(1,2)==2:
                                    comment_box.send_keys('Really good @'+username+' ! Check out my art account, just started it and am trying to get it off the ground!')
                                else:
                                    comment_box.send_keys('This is really good @'+username+', great work! Please check out my art profile, I just started it and am trying to get it off the ground! Thank you!!!')

                            sleep(1)
                            post_comment = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[3]/div/form/button')
                            post_comment.click()
                            try:
                                webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
                            except:
                                pass
                            sleep(randint(25,30))

                        webdriver.find_element_by_link_text('Next').click()
                        sleep(randint(25,30))
                                                              
                    elif likecount1 >= 1000:
                        button_like = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[1]/span[1]/button/span')       
                        button_like.click()
                        sleep(randint(5,10))
                        try:
                            webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
                        except:
                            pass
                        likes += 1
                        sleep(randint(25,30))
                        
                        webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[1]/span[2]/button/span').click()
                        comment_box = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[3]/div/form/textarea')
                        
                        if randint(1,10) < 6:
                            if randint(1,2)==2:
                                comment_box.send_keys('Awesome stuff @'+username+' ! Check out my profile, I am new to IG and am trying to get my page some viewership! Much thanks!!!')
                            else:                            
                                comment_box.send_keys('Really great work @'+username+' ! Please check out my profile, new to IG trying to get my page up off the ground! Much appreciated!!!')
                            sleep(1)
                            post_comment = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[3]/div/form/button')
                            post_comment.click()
                            try:
                                webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
                            except:
                                pass
                            comments += 1
                            sleep(randint(25,30))
                        else:
                            webdriver.find_element_by_link_text('Next').click()
                            sleep(randint(25,30))
                        
                        webdriver.find_element_by_link_text('Next').click()
                        sleep(randint(25,30))  
                    elif (likecount1 >= 10) & (likecount1 < 50):
                        if randint(1,2)==2:
                            if webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/header/div[2]/div[1]/div[2]/button').text == 'Follow':
                                webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/header/div[2]/div[1]/div[2]/button').click()
                                try:
                                    webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
                                except:
                                    pass
                                followed += 1
                        button_like = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[1]/span[1]/button/span')       
                        button_like.click()
                        sleep(randint(5,10))
                        try:
                            webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
                        except:
                            pass
                        likes += 1
                        sleep(randint(25,30))
                        
                        if randint(1,5)==3:
                            comment_box.send_keys('Awesome stuff @'+username+' !')
                        else:                            
                            comment_box.send_keys('Great work @'+username+' !')
                        sleep(1)
                        post_comment = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[3]/div/form/button')
                        post_comment.click()
                        try:
                            webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
                        except:
                            pass
                        comments += 1
                        sleep(randint(25,30))
                        
                        webdriver.find_element_by_link_text('Next').click()
                        sleep(randint(25,30))
                    else:
                        webdriver.find_element_by_link_text('Next').click()
                        sleep(randint(25,30))
                else:
                    webdriver.find_element_by_link_text('Next').click()
                    sleep(randint(25,30))
            
            except:
                webdriver.find_element_by_link_text('Next').click()
                sleep(randint(25,30))
                                                
        if  followed < 10:
            continue
        else:
            sleep(1800)
            followed = 0
    except:
        continue

for n in range(0,len(new_followed)):
    prev_user_list.append(new_followed[n])
for n in range(0,len(new_comment)):
    comment_list.append(new_comment[n])    

updated_user_df = pd.DataFrame(prev_user_list)
updated_user_df.to_csv(r'C:\Users\fitzg\users_followed_list.csv'.format())#strftime("%Y%m%d-%H%M%S")))
updated_comment = pd.DataFrame(comment_list)
updated_comment.to_csv(r'C:\Users\fitzg\comments_list.csv'.format())#strftime("%Y%m%d-%H%M%S")))

print('Liked {} photos.'.format(likes))
print('Commented {} photos.'.format(comments))
print('Followed {} new people.'.format(followed))

Liked 1 photos.
Commented 0 photos.
Followed 0 new people.


In [56]:
try:
    webdriver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[1]').click()
except:
    pass
    

In [65]:
#for interacting with own followers
for x in follower_list:
    webdriver.get('https://www.instagram.com/'+ x + '/')
    sleep(5)
        
    first_thumbnail = webdriver.find_element_by_class_name('_9AhH0')
    first_thumbnail.click()
    sleep(randint(1,2))    

    comment = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span').text
    
    if comment not in comment_list:
        
        new_comment.append(comment)
        button_like = webdriver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/div[2]/section[1]/span[1]/button/span')       
        button_like.click()
        sleep(randint(18,25))
        
        comm_prob = randint(1,10)
        if (comm_prob > 3) & (comm_prob < 7):
            comments += 1
            webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[1]/span[2]/button/span').click()
            comment_box = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[3]/div/form/textarea')

            if (comm_prob ==4):
                comment_box.send_keys('Nice!')
            elif comm_prob ==4:
                comment_box.send_keys('Great work!')
            elif comm_prob == 6:
                comment_box.send_keys('Amazing!')

            sleep(1)
            post_comment = webdriver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[3]/div/form/button')
            post_comment.click()
            sleep(randint(22,28))

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/div/article/div[2]/section[1]/span[1]/button/span"}
  (Session info: chrome=76.0.3809.100)
